In [1]:
# imports
import numpy as np
import cv2
from PIL import Image
from scipy.fftpack import dct
from scipy.fftpack import idct
from math import log10, sqrt

In [2]:
# Specify image files here
input_image = 'cover.jpg'
watermark_logo = 'iitbbs_logo.jpeg'
output_image = 'watermarked_image.png'
recovered_logo = 'recovered_watermark.png'

In [3]:
# Convert Image to grayscale and generate image array
def convert_image(image_name, size):   
  img = Image.open(image_name).resize((size, size), 1)  
  img = img.convert('L')  
  img.save(image_name)
  image_array = np.array(img.getdata(), dtype = float).reshape((size, size))              
  return image_array

In [4]:
# Discrete Cosine Transform
def apply_dct(image_array): 
  size = image_array[0].__len__()  
  all_subdct = np.empty((size, size))  
  
  for i in range (0, size, 8):  
    for j in range (0, size, 8):          
      subpixels = image_array[i:i+8, j:j+8]         
      subdct = dct(dct(subpixels.T, norm="ortho").T, norm="ortho")         
      all_subdct[i:i+8, j:j+8] = subdct

  return all_subdct

In [5]:
# Embed watermark in given image
def embed_watermark(watermark_array, orig_image):   
  watermark_array_size = watermark_array[0].__len__()    
  watermark_flat = watermark_array.ravel()    
  ind = 0
  
  for x in range (0, orig_image.__len__(), 8):      
    for y in range (0, orig_image.__len__(), 8):        
      if ind < watermark_flat.__len__():           
        subdct = orig_image[x:x+8, y:y+8]             
        subdct[5][5] = watermark_flat[ind]              
        orig_image[x:x+8, y:y+8] = subdct            
        ind += 1 

  return orig_image

In [6]:
# Inverse DCT
def inverse_dct(all_subdct):   
  size = all_subdct[0].__len__()   
  all_subidct = np.empty((size, size))  

  for i in range (0, size, 8):   
      for j in range (0, size, 8):    
        subidct = idct(idct(all_subdct[i:i+8, j:j+8].T, norm="ortho").T, norm="ortho")      
        all_subidct[i:i+8, j:j+8] = subidct
        
  return all_subidct

In [7]:
# Recoved watermark
def recover_watermark(watermarked_image_array, output_logo):
  dct_watermarked_image_array = apply_dct(watermarked_image_array)  

  watermark_size = 128   
  subwatermarks = []

  for x in range (0, dct_watermarked_image_array.__len__(), 8):     
    for y in range (0, dct_watermarked_image_array.__len__(), 8):           
      coeff_slice = dct_watermarked_image_array[x:x+8, y:y+8]        
      subwatermarks.append(coeff_slice[5][5])
      
  watermark = np.array(subwatermarks).reshape(watermark_size, watermark_size)
  watermark =  np.uint8(watermark)

  img = Image.fromarray(watermark)
  img.save(output_logo)

In [8]:
# Calculate PSNR
def PSNR(original, compressed): 
    mse = np.mean((original - compressed) ** 2) 
    if(mse == 0):  
        return 100
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse)) 
    return psnr

In [9]:
# Correlation coefficient
def correlation_coefficient(patch1, patch2):
    product = np.mean((patch1 - patch1.mean()) * (patch2 - patch2.mean()))
    stds = patch1.std() * patch2.std()

    if stds == 0:
        return 0
    else:
        product /= stds
        return product

## Run here to put watermark on image

In [10]:
# converting images to greyscale
image_array = convert_image(input_image, 1024)  
watermark_array = convert_image(watermark_logo, 128)

# applying dct and embedding watermark on image
dct_array = apply_dct(image_array)   
dct_array = embed_watermark(watermark_array, dct_array)   
watermarked_image_array = inverse_dct(dct_array)

# save watermarked image from array
image_array_copy = watermarked_image_array.clip(0, 255)   
image_array_copy = image_array_copy.astype("uint8")  
img = Image.fromarray(image_array_copy)  
img.save(output_image)
print("Watermark", watermark_logo, "put on", input_image, ". Image saved as", output_image)

Watermark iitbbs_logo.jpeg put on cover.jpg . Image saved as watermarked_image.png


In [11]:
# Calculate PSNR between original image and watermarked image
original_image = cv2.imread(input_image)
watermarked_image = cv2.imread(output_image) 
psnr = PSNR(original_image, watermarked_image)
print("PSNR =", psnr)

PSNR = 29.234398346866772


In [12]:
# Recover watermark from image
recover_watermark(watermarked_image_array, recovered_logo)
print("Recovered logo =", recovered_logo)

Recovered logo = recovered_watermark.png


In [13]:
# Calculate Correlation coefficient between original logo and recovered logo
img1 = convert_image(watermark_logo, 128)
img2 = convert_image(recovered_logo, 128)
corr_coeff = correlation_coefficient(img1, img2)
print("Correlation Coefficient =", corr_coeff)

Correlation Coefficient = 0.999808452197669


# Attacks

### Rotational Attack

In [14]:
img = cv2.imread(output_image)
img = cv2.cvtColor(np.float32(img), cv2.COLOR_BGR2GRAY)
center = (512, 512)
rotation_matrix = cv2.getRotationMatrix2D(center, 90, 1)
tampered_image = cv2.warpAffine(img, rotation_matrix, (1024, 1024))
cv2.imwrite('rotational_attack.png', tampered_image)
print("Rotational attack done", 'rotational_attack.png')

Rotational attack done rotational_attack.png


In [15]:
# Recover watermark from image
recovered_logo = 'rotational_attack_recovered_watermark.png'
recover_watermark(tampered_image, recovered_logo)
print("Recovered logo =", recovered_logo)

Recovered logo = rotational_attack_recovered_watermark.png


In [16]:
# Calculate Correlation coefficient between original logo and recovered logo
img1 = convert_image(watermark_logo, 128)
img2 = convert_image(recovered_logo, 128)
corr_coeff = correlation_coefficient(img1, img2)
print("Correlation Coefficient =", corr_coeff)

Correlation Coefficient = 0.287190645647429


### Average Filer

In [17]:
img = cv2.imread(output_image, 0)

m = img.shape[0]
n = img.shape[1]

filter = np.ones([3,3], dtype = int)
filter = filter/9

tampered_image = np.zeros([m,n])

for i in range(1, m-1):
  for j in range(1, n-1):
      temp = (img[i-1, j-1]*filter[0, 0])+(img[i-1, j]*filter[0, 1])+(img[i-1, j + 1]*filter[0, 2])+(img[i, j-1]*filter[1, 0])+ (img[i, j]*filter[1, 1])+(img[i, j + 1]*filter[1, 2])+(img[i + 1, j-1]*filter[2, 0])+(img[i + 1, j]*filter[2, 1])+(img[i + 1, j + 1]*filter[2, 2])         
      tampered_image[i,j] = temp

cv2.imwrite('average_filter_attack.png', tampered_image)
print("Average filter attack done", 'average_filter_attack.png')

Average filter attack done average_filter_attack.png


In [18]:
# Recover watermark from image
recovered_logo = 'average_filter_attack_recovered_watermark.png'
recover_watermark(tampered_image, recovered_logo)
print("Recovered logo =", recovered_logo)

Recovered logo = average_filter_attack_recovered_watermark.png


In [19]:
# Calculate Correlation coefficient between original logo and recovered logo
img1 = convert_image(watermark_logo, 128)
img2 = convert_image(recovered_logo, 128)
corr_coeff = correlation_coefficient(img1, img2)
print("Correlation Coefficient =", corr_coeff)

Correlation Coefficient = -0.0072237797838706624


### Median Filter

In [20]:
img = cv2.imread(output_image, 0)
m, n = img.shape

tampered_image = np.zeros([m, n])
  
for i in range(1, m-1):
    for j in range(1, n-1):
        temp = [img[i-1, j-1],
              img[i-1, j],
              img[i-1, j + 1],
              img[i, j-1],
              img[i, j],
              img[i, j + 1],
              img[i + 1, j-1],
              img[i + 1, j],
              img[i + 1, j + 1]]
          
        temp = sorted(temp)
        tampered_image[i, j] = temp[4]
  
tampered_image = tampered_image.astype(np.uint8)
cv2.imwrite('median_filter_attack.png', tampered_image)
print("Median filter attack done", 'median_filter_attack.png')

Median filter attack done median_filter_attack.png


In [21]:
# Recover watermark from image
recovered_logo = 'median_filter_attack_recovered_watermark.png'
recover_watermark(tampered_image, recovered_logo)
print("Recovered logo =", recovered_logo)

Recovered logo = median_filter_attack_recovered_watermark.png


In [22]:
# Calculate Correlation coefficient between original logo and recovered logo
img1 = convert_image(watermark_logo, 128)
img2 = convert_image(recovered_logo, 128)
corr_coeff = correlation_coefficient(img1, img2)
print("Correlation Coefficient =", corr_coeff)

Correlation Coefficient = -0.09738330858842437


### Cropping Attack

In [23]:
img = Image.open(output_image)
tampered_image = img.crop( (1, 0, 1022, 1023) )
tampered_image.save('cropping_attack.png')
print("Cropping attack done", 'cropping_attack.png')

Cropping attack done cropping_attack.png


In [24]:
# Recover watermark from image
recovered_logo = 'cropping_attack_recovered_watermark.png'
img = convert_image("cropping_attack.png", 1024)
recover_watermark(img, recovered_logo)
print("Recovered logo =", recovered_logo)

Recovered logo = cropping_attack_recovered_watermark.png


In [25]:
# Calculate Correlation coefficient between original logo and recovered logo
img1 = convert_image(watermark_logo, 128)
img2 = convert_image(recovered_logo, 128)
corr_coeff = correlation_coefficient(img1, img2)
print("Correlation Coefficient =", corr_coeff)

Correlation Coefficient = 0.15508941317380637
